# Stochastic Red Blue Set Covering 

Experiments for Approximation Algorithms 

In [1]:
import gurobipy as gp
import pandas as pd
import numpy as np
from gurobipy import GRB
from itertools import product
import math, sys, time
from netgraph import Graph, InteractiveGraph, EditableGraph
import matplotlib.pyplot as plt
import multiprocessing
import networkx as nx
import random
import pickle as pkl

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:

#web license try to access it via uoft
e = gp.Env("gurobi.log")

Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2025-02-13


In [3]:
from rbsc import *

In [4]:
def defineinstance(N,n_elem, Scenarios, maxBlueProb, TransmissionProb, CoverFactor, N_nodes, lambd, Plot = False):
  Sets = {}
  n=np.arange(n_elem)+1
  # Adapated from https://stackoverflow.com/questions/71024509/create-different-disconnected-graphs-from-a-set-of-fixed-nodes-in-networkx
  #randomly permuting nodes
  Elements=np.random.permutation(n)

  N_graphs= round(math.sqrt(n_elem)/2)
  #assign the random modes to each graph
  random_graphs_nodes=[Elements[N_nodes*i:N_nodes*(i+1)] for i in range(N_graphs)]

  #create random graphs
  r_g=[nx.erdos_renyi_graph(n=N_nodes,p=0.5) for _ in range(N_graphs)]

  #relabel the nodes in each graph according to random_graphs_nodes
  mappings=[]
  for i in range(N_graphs):
    mappings.append({j:random_graphs_nodes[i][j] for j in range(N_nodes)})
    r_g[i]=nx.relabel_nodes(r_g[i], mappings[i]) 

  if Plot:
    #plot result
    fig=plt.figure(figsize=(15,6))
    for i in range(N_graphs):
      plt.subplot(1,N_graphs,i+1)
      plt.xlabel('Graph '+str(i+1))
      plt.tight_layout()
      # nx.draw(r_g[i],pos=pos,with_labels=True,node_color=colors[i])
      g = Graph(r_g[i],node_labels=True, node_layout = 'spring',
          node_label_fontdict=dict(size=10), node_label_offset=0.05, node_size=3, edge_width=0.4)
      plt.savefig("UncertaintyGraph.svg")
  NodeProbability = {e:np.random.random()*maxBlueProb for e in Elements}
  RedScenarios = {}
  BlueScenarios = {}
  for xi in range(Scenarios):
    Blues_xi = {}
    Reds_xi = {}
    for e in Elements:
      if np.random.random() <= NodeProbability[e]:
        Blues_xi[e] = 'B'
    if Blues_xi == {}:
      Blues_xi[e] = 'B'
    InitialBlues= list(Blues_xi.keys())
    for b in InitialBlues:
      for i in range(N_graphs):
        if b in r_g[i].nodes():
          #propagate the fraud
          infectedByb = []
          propagate(r_g[i],b,infectedByb, TransmissionProb)
          for infected in infectedByb:
            Blues_xi[infected] = 'B'
    Reds_xi = {e:'R' for e in Elements if e not in Blues_xi.keys()}
    BlueScenarios[xi] = Blues_xi
    RedScenarios[xi] = Reds_xi

    for xi in BlueScenarios.keys():#test 
      if set(BlueScenarios[xi]).union(set(RedScenarios[xi])) != set(Elements):
        print("Something is wrong")

  #form the extensive problem 
  ExtensiveBlues = {(b, xi):'B' for xi in BlueScenarios.keys() for b in BlueScenarios[xi].keys()}
  #blues are all combinations of xi and blue elements 
  #reds are all combinations of xi and red elements + one red for each x, blue pair with weight lambda 
  #sets are the given sets plus the super sets for each xi, blue element pair
  ExtensiveReds = {(r, xi): 'R' for xi in RedScenarios.keys() for r in RedScenarios[xi].keys()}
  ExtensiveRedsWeights = {(r, xi): 1/Scenarios for xi in RedScenarios.keys() for r in RedScenarios[xi].keys()}

  BlueMapping = {}
  LastElement = max(Elements)
  k = 1
  for (b, xi) in ExtensiveBlues.keys():
    BlueMapping[(b, xi)] = (LastElement + k, xi)
    ExtensiveReds[(LastElement + k, xi)] = 'Super'
    ExtensiveRedsWeights[(LastElement + k, xi)] = lambd/Scenarios
    k = k + 1

  #form first stage sets that cover elements that are ever blue
  k = 0
  EverBlues = set(b for (b,xi) in ExtensiveBlues.keys())
  UncoveredBlues = [b for b in EverBlues]
  #randomly sample until all the blues are covered 
  while (k < N or UncoveredBlues != []):
    n_k = random.sample(range(1,(1+len(Elements))//CoverFactor),1)[0]
    DoesNotCoverAnyBlue = True
    while DoesNotCoverAnyBlue:
      SetCandidate = random.sample(list(Elements), n_k)
      BlueCoveredBool = [b in SetCandidate for b in EverBlues]
      if True in BlueCoveredBool:
        DoesNotCoverAnyBlue = False
    if UncoveredBlues == []:
      ind = 'Set' + str(k)
      Sets[ind] = SetCandidate
      k = k+1

    CoversNewBlue = False
    for element in EverBlues:
      if element in SetCandidate and element in UncoveredBlues:
        UncoveredBlues.remove(element)
        CoversNewBlue = True
    if CoversNewBlue:
      ind = 'Set' + str(k)
      Sets[ind] = SetCandidate
      k = k+1
  #element scenario pairs 
  ExtensiveFormElements = set(product(Elements, range(Scenarios)))

  ScenarioFormSets = {}

  #appending the base elements for each scenario
  for S in Sets.keys():
    ScenarioFormSets[S] = set()
    for e, xi in ExtensiveFormElements:
      probability_swap = 0.1
      if e in Sets[S] and probability_swap <= np.random.random():
        ScenarioFormSets[S].add((e,xi))
      else:
        RandomSet = random.choice(list(Sets.keys())) #pick new set
        try:
          ScenarioFormSets[RandomSet].add((e,xi))
        except:
          ScenarioFormSets[RandomSet] = []

  #appending the super sets
  SuperLookup = {}
  bLookup = {}
  k = 0
  for (b, xi) in ExtensiveBlues:
    (r, xi2) = BlueMapping[(b,xi)]
    ScenarioFormSets['Super'+str(k)] = [(b, xi), (r, xi)]
    RedScenarios[xi][r] = 'R' #include the new red elements here
    SuperLookup[(b, xi)] =  'Super'+str(k)
    bLookup[('Super'+str(k), xi)] = b
    k += 1

  SetsIndexedbyScenario = {}
  for xi in range(Scenarios):
    SetsinScenario = {}
    for S in ScenarioFormSets.keys():
      Temp = []
      for pair in ScenarioFormSets[S]:
        if pair[1] == xi: 
          Temp.append(pair[0])
      if Temp != []:
        SetsinScenario[S] = Temp
    SetsIndexedbyScenario[xi] = SetsinScenario
  SetsinScenario

  #getting the weights indexed by scenario
  WeightsIndexedbyScenario = {}
  for xi in range(Scenarios):
    WeightsinScenario = {}
    for (r, xi_2), wgt in ExtensiveRedsWeights.items():
      if xi_2 == xi:
        WeightsinScenario[r] = wgt
    WeightsIndexedbyScenario[xi] = WeightsinScenario
    
  return Sets, ExtensiveReds, ExtensiveBlues, ScenarioFormSets, ExtensiveRedsWeights, SetsIndexedbyScenario, RedScenarios, BlueScenarios, WeightsIndexedbyScenario, SuperLookup, bLookup, BlueMapping

## Experiments

In [5]:
#Results = {}
n_elem = 30 #number of elements 
maxBlueProb = 0.1 #
TransmissionProb = 0.9
mipgaptol = 0.05
global LIMIT #time limit
LIMIT = 2*60*60
CoverFactor = 1
N_nodes=4
output = False
random.seed(10)

M = 15 #replications 

trial = 0

AverageNumberofReds = {}

set_sizes = [3, 5, 10, 15, 20]

scenario_sizes = [5, 10, 20, 40]*M #for the replications

multiplier = 2

In [114]:


for N, Scenarios in product(set_sizes, scenario_sizes):
#for N, Scenarios in product([5], [10]*M + [20]*M + [30]*M + [40]*M):

#for N, Scenarios in product([10, 40], [5]):
  print("Trial # ", trial)
  lambd = 25*(n_elem)/np.log(1+N) #average elements in set?

  Sets = {}

  #random scenario generation
  #define each element to be a part of a 
  #graph. With edge probability p 
  #create instance 

  start = time.time()
  (Sets, ExtensiveReds, ExtensiveBlues, ScenarioFormSets, 
  ExtensiveRedsWeights, SetsIndexedbyScenario, RedScenarios, 
  BlueScenarios, WeightsIndexedbyScenario, SuperLookup, 
  bLookup, BlueMapping) = defineinstance(N,n_elem, Scenarios,
                                          maxBlueProb, TransmissionProb, 
                                          CoverFactor, N_nodes, lambd, Plot = False)

  
  AverageNumberofReds[(N, Scenarios, trial)] = get_average_number_of_reds(RedScenarios, Sets)

  trial = trial + 1
  

Trial #  1201
average # of reds 19.866666666666667
Trial #  1202
average # of reds 14.74
Trial #  1203
average # of reds 15.916666666666666
Trial #  1204
average # of reds 11.1875
Trial #  1205
average # of reds 16.0
Trial #  1206
average # of reds 14.925
Trial #  1207
average # of reds 15.24
Trial #  1208
average # of reds 15.91
Trial #  1209
average # of reds 14.666666666666666
Trial #  1210
average # of reds 20.1
Trial #  1211
average # of reds 20.416666666666668
Trial #  1212
average # of reds 13.1125
Trial #  1213
average # of reds 8.2
Trial #  1214
average # of reds 11.98
Trial #  1215
average # of reds 19.15
Trial #  1216
average # of reds 15.49
Trial #  1217
average # of reds 20.133333333333333
Trial #  1218
average # of reds 12.433333333333334
Trial #  1219
average # of reds 13.2
Trial #  1220
average # of reds 15.21875
Trial #  1221
average # of reds 22.2
Trial #  1222
average # of reds 19.866666666666667
Trial #  1223
average # of reds 17.316666666666666
Trial #  1224
averag

In [115]:
AverageNumberofRedsSeries = pd.Series(AverageNumberofReds)
AverageNumberofRedsSeries.index = AverageNumberofRedsSeries.index.set_names(['NSets', 'NScenarios', 'Trial'])
AverageNumberofRedsSeriesGrouped = AverageNumberofRedsSeries.groupby(['NSets', 'NScenarios']).mean()
AverageNumberofRedsSeriesGrouped.to_pickle("data/average_reds.pkl")

In [116]:
AverageNumberofRedsSeriesGrouped = pd.read_pickle("data/average_reds.pkl")

In [117]:
from IPython.display import clear_output

#trial = 0
multiplier = 2

for N, Scenarios in product(set_sizes, scenario_sizes):
#for N, Scenarios in product([5], [10]*M + [20]*M + [30]*M + [40]*M):
  print("Trial # ", trial)
#for N, Scenarios in product([10, 40], [5]):
  lambd = multiplier*AverageNumberofRedsSeriesGrouped[(N,Scenarios)] #average elements in set

  Sets = {}

  #random scenario generation
  #define each element to be a part of a 
  #graph. With edge probability p 
  #create instance 
  (Sets, ExtensiveReds, ExtensiveBlues, ScenarioFormSets, 
  ExtensiveRedsWeights, SetsIndexedbyScenario, RedScenarios, 
  BlueScenarios, WeightsIndexedbyScenario, SuperLookup, 
  bLookup, BlueMapping) = defineinstance(N,n_elem, Scenarios,
                                          maxBlueProb, TransmissionProb, 
                                          CoverFactor, N_nodes, lambd, Plot = False)

  ##exact soln alg on the original problem
  (SelectedReds, SelectedSets, 
  SolnEdges, ObjVal, ObjBound, RunTime) = DeterministicRedBlue(ExtensiveReds, ExtensiveBlues, 
                                                                ScenarioFormSets, ExtensiveRedsWeights, 
                                                                LIMIT, output = True, testing = True, 
                                                               mipgap = mipgaptol, 
                                                               env = e)

  Results[('Extensive', trial, n_elem,N,Scenarios)] = [ObjVal, ObjBound, RunTime]  #add to the results dictionary
  
  # peleg algorithm 
  start = time.time()
  BestCover, MinWgt = LowDeg2(ExtensiveReds, ExtensiveBlues, ScenarioFormSets, ExtensiveRedsWeights)
  end = time.time()
  RedsinCandidates = ElementsinFamily(BestCover, ExtensiveReds)
  print("Peleg ", NumRedsinS(RedsinCandidates, ExtensiveReds, ExtensiveRedsWeights))
  if MinWgt > 1000:
    print("Best Sets ", BestCover.keys())
  Results[('Peleg', trial, n_elem,N,Scenarios)] = [MinWgt, 0 , end-start]   #add to the results dictionary

  
  ##approximation alg using LP on the original problem 
  # ##approximation alg using LP on the augmented problem
  # AugScenarioFormSets, b_count, smallest_augmented_sets_containingb = FormAugmentedProblem(ExtensiveReds, ExtensiveBlues, ScenarioFormSets)

  AugScenarioFormSets = FormAugmentedProblem(ExtensiveReds, ExtensiveBlues, ScenarioFormSets)

  start = time.time()
  (SelectedReds, SelectedSets, 
  SolnEdges, vals_y, vals_x) = DeterministicRedBlueAugmented(ExtensiveReds, ExtensiveBlues, 
                                                                                        AugScenarioFormSets, Relax=True, Weights=ExtensiveRedsWeights, output=True, q=None, env=e)
  
  SelectedReds, SelectedAugSets, SolnEdges = CarrApproximationAlgorithm(ExtensiveReds, 
                                                  ExtensiveBlues, 
                                                  AugScenarioFormSets, 
                                                  vals_y)
  end = time.time()
  Wgt = sum(ExtensiveRedsWeights[t] for t in SelectedReds)

  print("Carr ", Wgt)
  Results[('Carr', trial, n_elem, N, Scenarios)] = [Wgt, 0 , end-start] #add to results dictionary

  #vals = m.getAttr('x', x)
  trial = trial + 1
  
  print("Completed instance defined by Nelem, N, Scenarios", (n_elem,N,Scenarios))
  with open("data/appx_results_d2_2023_"+ str(N)+".pkl", 'wb') as fp:
    pkl.dump(Results, fp);
    pd.DataFrame(Results).T.to_csv("results.csv")
    
  clear_output(wait = False)

Trial #  1580
Set parameter MIPGap to value 0.05
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 4390 rows, 1312 columns and 9097 nonzeros
Model fingerprint: 0x0abc1124
Variable types: 1200 continuous, 112 integer (112 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 28.7059870
Presolve removed 4211 rows and 1229 columns
Presolve time: 0.01s
Presolved: 179 rows, 83 columns, 443 nonzeros
Found heuristic solution: objective 27.2000000
Variable types: 0 continuous, 83 integer (83 binary)

Root relaxation: objective 1.370399e+01, 69 iterations, 0.00 seconds (0.00 work units)
    Nodes  

KeyboardInterrupt: 

# Results

* Import the pickle file for the results 
* Calculate the optimality gap 
* Calculate run-time statistics

In [78]:
pd.DataFrame(Results).T

0      1          2
Extensive 0   10 3  5    7.200000  7.200   0.009000
Peleg     0   10 3  5   19.135911  0.000   0.001034
Carr      0   10 3  5    7.200000  0.000   0.034532
Extensive 1   10 3  10   7.400000  7.400   0.022000
Peleg     1   10 3  10  11.595556  0.000   0.002050
...                           ...    ...        ...
          298 10 20 20   7.494050  0.000   0.010000
Carr      298 10 20 20  11.364300  0.000   3.756073
Extensive 299 10 20 40   7.275000  7.275   0.836000
Peleg     299 10 20 40   7.378938  0.000   0.038525
Carr      299 10 20 40  10.276300  0.000  83.863640

[900 rows x 3 columns]

In [7]:
# with open("data/appx_results_final.pkl", 'rb') as f:
#       Results = pkl.load(f);
# 
# out = pd.DataFrame(Results, index = ['Objective', 'Bound', 'Time']).transpose()
# out.index = out.index.set_names(['Type', 'Trial', 'NElem', 'NSets', 'NScenarios'])
# pd.concat([out]).to_pickle("data/appx_results_final.pkl")

In [8]:
out = pd.read_pickle("data/appx_results_final.pkl")

In [9]:
out.groupby(level = ['Type','NElem', 'NSets', 'NScenarios']).median().unstack(level=0)

Objective                      Bound                  \
Type                         Carr Extensive      Peleg  Carr Extensive Peleg   
NElem NSets NScenarios                                                         
10    3     5            7.200000  7.200000  15.926756   0.0  7.133333   0.0   
            10           7.450000  7.450000  14.253111   0.0  7.450000   0.0   
            20           7.950000  7.925000  17.203056   0.0  7.873889   0.0   
            40           8.050603  8.050603  17.458786   0.0  8.012500   0.0   
      5     5            6.896800  6.700000   9.293600   0.0  6.700000   0.0   
            10           7.400000  7.300000   8.211867   0.0  7.200000   0.0   
            20           7.670867  7.675000  15.392722   0.0  7.584056   0.0   
            40           7.783300  7.783300  16.525983   0.0  7.505533   0.0   
      10    5            6.790667  5.700000   7.000000   0.0  5.561333   0.0   
            10           6.700000  6.600000   7.258667   0.0  6.425333   0.0   
            20           7.375000  7.051600   7.828200   0.0  6.853200   0.0   
            40           8.591225  7.487500   7.971367   0.0  7.212500   0.0   
      15    5            7.004089  5.100000   5.900000   0.0  5.000000   0.0   
            10           7.700000  6.466889   7.200000   0.0  6.250000   0.0   
            20           8.491833  6.944556   7.441833   0.0  6.748639   0.0   
            40          10.383944  7.375000   7.881794   0.0  7.148222   0.0   
      20    5            6.545600  4.600000   5.500000   0.0  4.433600   0.0   
            10           7.487667  6.050000   6.700000   0.0  5.950000   0.0   
            20           9.188100  6.725000   7.316075   0.0  6.508037   0.0   
            40          10.171694  7.248681   7.712500   0.0  7.200000   0.0   

                              Time                      
Type                          Carr Extensive     Peleg  
NElem NSets NScenarios                                  
10    3     5             0.034532    0.0130  0.001008  
            10            0.206850    0.0150  0.002005  
            20            2.376813    0.0125  0.004717  
            40           26.099716    0.0150  0.015196  
      5     5             0.055042    0.0160  0.001000  
            10            0.280785    0.0210  0.002034  
            20            3.326906    0.0230  0.007471  
            40           43.575071    0.0250  0.030518  
      10    5             0.079244    0.0325  0.001002  
            10            0.484657    0.0370  0.002503  
            20            5.236489    0.0605  0.010521  
            40           88.485649    0.0890  0.042356  
      15    5             0.097976    0.0450  0.002000  
            10            0.633476    0.0780  0.004006  
            20            9.989918    0.1520  0.015558  
            40          124.778257    0.3385  0.039627  
      20    5             0.120192    0.0440  0.002001  
            10            0.976068    0.1085  0.004499  
            20           12.405256    0.2330  0.017272  
            40          164.330716    0.8510  0.053504

In [103]:
out.to_csv("data/APX2023.csv")

In [10]:
Extensive = out[out.index.get_level_values('Type') == 'Extensive'].droplevel("Type")
Peleg = out[out.index.get_level_values('Type') == 'Peleg'].droplevel("Type")
Carr = out[out.index.get_level_values('Type') == 'Carr'].droplevel("Type")

In [11]:
Peleg_Gap = 100*(Peleg.Objective - Extensive.Objective )/Extensive.Objective
Carr_Gap = 100*(Carr.Objective - Extensive.Objective )/Extensive.Objective
# Carr_Gap= pd.DataFrame(Carr_Gap).reset_index()
# Peleg_Gap = pd.DataFrame(Peleg_Gap).reset_index()

In [16]:
peleg_median = pd.DataFrame(Peleg_Gap.groupby(level = ['NElem', 'NSets', 'NScenarios']).median())
peleg_median.columns = ["Peleg"]
carr_median = pd.DataFrame(Carr_Gap.groupby(level = ['NElem', 'NSets', 'NScenarios']).median())
carr_median.columns = ["Carr et al."]
pd.concat([peleg_median, carr_median], axis = 1).stack().unstack(1).unstack(-1).to_csv("data/median_gap.csv")

In [17]:
peleg_median = pd.DataFrame(Peleg_Gap.groupby(level = ['NElem', 'NSets', 'NScenarios']).quantile(0.9))
peleg_median.columns = ["Peleg"]
carr_median = pd.DataFrame(Carr_Gap.groupby(level = ['NElem', 'NSets', 'NScenarios']).quantile(0.9))
carr_median.columns = ["Carr et al."]
pd.concat([peleg_median, carr_median], axis = 1).stack().unstack(1).unstack(-1).to_csv("data/90_gap.csv")

In [18]:
peleg_median = pd.DataFrame(Peleg.Time.groupby(level = ['NElem', 'NSets', 'NScenarios']).median())
peleg_median.columns = ["Peleg"]
carr_median = pd.DataFrame(Carr.Time.groupby(level = ['NElem', 'NSets', 'NScenarios']).median())
carr_median.columns = ["Carr et al."]
extensive_median = pd.DataFrame(Extensive.Time.groupby(level = ['NElem', 'NSets', 'NScenarios']).median())
extensive_median.columns = ["Exact Solve"]
pd.concat([peleg_median, carr_median, extensive_median], axis = 1).stack().unstack(1).unstack(-1).to_csv("data/median_time.csv")

In [19]:
peleg_median = pd.DataFrame(Peleg.Time.groupby(level = ['NElem', 'NSets', 'NScenarios']).quantile(0.9))
peleg_median.columns = ["Peleg"]
carr_median = pd.DataFrame(Carr.Time.groupby(level = ['NElem', 'NSets', 'NScenarios']).quantile(0.9))
carr_median.columns = ["Carr et al."]
extensive_median = pd.DataFrame(Extensive.Time.groupby(level = ['NElem', 'NSets', 'NScenarios']).quantile(0.9))
extensive_median.columns = ["Exact Solve"]
pd.concat([peleg_median, carr_median, extensive_median], axis = 1).stack().unstack(1).unstack(-1).to_csv("data/90_time.csv")